In [ ]:
!pip3 install git+https://github.com/sinzlab/neuralpredictors.git

In [ ]:
!pip3 install scikit-image

In [ ]:
!pip3 install ipywidgets


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
from utils import download_with_requests, set_background
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from tqdm.notebook import trange, tqdm
from digital_twin_library import SubsetSampler, corr, PoissonLoss
import seaborn as sns

from torch.utils.data import DataLoader

from neuralpredictors.data.datasets import FileTreeDataset

from neuralpredictors.data.transforms import (
    ToTensor,
    NeuroNormalizer,
    ScaleInputs,
)

# Data

Let's download some data from the sensorium competition 2022. You can check out all datasets [here]("https://gin.g-node.org/cajal/Sensorium2022).

In [ ]:
!unzip -n -q /datasets/day01/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip -d sensorium_data/


Let's look at the content of the data quickly. It contains images shown to a mouse, responses of cell in primary visual cortex (deconvolved Calcium activity), behavior of the animal (running and pupil dilation), and the location of the pupil center in an eye tracking video.

In [ ]:
ls sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6/data


In [ ]:
ls sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6/data/images/100*.npy

In [ ]:
img = np.load('sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6/data/images/0.npy')
plt.imshow(img.squeeze(), cmap='gray')
plt.axis('off')
img.shape

In [ ]:
responses = np.load('sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6/data/responses/0.npy')
responses.shape

The data contains gray scale images of size `144 x 256` and the corresponding responses of `8372` neurons in mouse primary visual cortex. For now, we'll ignore the behavior and the pupil and only use images and neuronal responses.

We can build a pytorch dataset for this data, using the the `FileTreeDataset` of `neuralpredictors`.

In [ ]:
from neuralpredictors.data.datasets import FileTreeDataset

root_dir = 'sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6'
dat = FileTreeDataset(root_dir, 'images', 'responses')
dat

Together with a sampler, we can create a pytorch dataloader

In [ ]:

train_sampler = SubsetSampler(dat.trial_info.tiers == 'train', shuffle=True)
train_loader = DataLoader(dat, sampler=train_sampler, batch_size=64)


Now we can iterate through this data loader

In [ ]:
for images, responses in train_loader:
    print(images.shape, responses.shape, type(images))
    print(f"Image mean+-std={images.mean()}+-{images.std()}")
    print(f"Reponses mean+-std={responses.mean()}+-{responses.std()}")
    break # let's stop after the first iteration

For our purpose here, the images are still a bit big (training time), so we let's downscale them by a factor of 4. In addition, let's standardize the pixel values and scale the neuronal responses to have std=1. We can do that with the help of a few functions in `neuralpredictors`.

In [ ]:


root_dir = 'sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6'
dat = FileTreeDataset(root_dir, 'images', 'responses')

transforms = [NeuroNormalizer(dat), ScaleInputs(scale=0.25), ToTensor(torch.cuda.is_available())]
dat.transforms.extend(transforms)


train_sampler = SubsetSampler(dat.trial_info.tiers == 'train', shuffle=True)
train_loader = DataLoader(dat, sampler=train_sampler, batch_size=64)


In [ ]:
for images, responses in train_loader:
    print(images.shape, responses.shape, type(images))
    print(f"Image mean+-std={images.mean()}+-{images.std()}")
    print(f"Reponses mean+-std={responses.mean()}+-{responses.std()}")
    break # let's stop after the first iteration

# Train our first model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root_dir = 'sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6'
dat = FileTreeDataset(root_dir, 'images', 'responses')

transforms = [ScaleInputs(scale=0.125), ToTensor(torch.cuda.is_available())]
dat.transforms.extend(transforms)

train_sampler = SubsetSampler(dat.trial_info.tiers == 'train', shuffle=True)
test_sampler = SubsetSampler(dat.trial_info.tiers == 'test', shuffle=False)
val_sampler = SubsetSampler(dat.trial_info.tiers == 'validation', shuffle=False)

train_loader = DataLoader(dat, sampler=train_sampler, batch_size=64)
val_loader = DataLoader(dat, sampler=val_sampler, batch_size=64)
test_loader = DataLoader(dat, sampler=test_sampler, batch_size=64)

print(f"The training device is: {device}")

<div class="task">

</div>

## First model

Now we are ready to build our first model. Please go through the tasks in order and see how/whether the performance of the model improves

1. Below you find a simple linear model. Write code to train the model on the training data.
    * What kind of loss could you use?
    * Monitor the performance (correlation) on the validation all 5 epochs (epoch=full sweep through the training data).
1. You'll find that the model is slow and does not train very well. Now try the following improvements and see how they increase the prediction performance.
    * Decrease the image resolution to `18 x 32`.
    * Add a `nn.BatchNorm1d` layer before the linear layer (make sure to put the model into `train` and `eval` mode respectively).
    * Add `weight_decay=1e-4` to the optimizer.
    * Add a rectifying nonlinearity `ReLU`, `Softplus` or `ELU + 1`.
    * Change the loss to `digital_twin_library.PoissonLoss`.
1. For the best model, produce plots that
    * compare the performance (correlation) between validation and train for all neurons.
    * plot linear filters (as grayscale images) of the best `n=12` neurons. What would you expect to see?
1. Think about the following questions
    * How many parameters does the model have?
    * Is there any benefit from having many neurons?
    * **Advanced**: The Poisson loss strictly a "correct" loss function?

In [ ]:
class GLM(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten the images
        return self.linear(x)

# Improving the model

Let's improve the model. First, let's use resolution `36 x 64` from now on. Also, let's standardize the images to `mean+-std=0+-1` and the neuronal responses to `std=1` (we keep them positive).

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root_dir = 'sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6'
dat = FileTreeDataset(root_dir, 'images', 'responses')

transforms = [NeuroNormalizer(dat), ScaleInputs(scale=0.25), ToTensor(torch.cuda.is_available())]
dat.transforms.extend(transforms)

train_sampler = SubsetSampler(dat.trial_info.tiers == 'train', shuffle=True)
test_sampler = SubsetSampler(dat.trial_info.tiers == 'test', shuffle=False)
val_sampler = SubsetSampler(dat.trial_info.tiers == 'validation', shuffle=False)

train_loader = DataLoader(dat, sampler=train_sampler, batch_size=64)
val_loader = DataLoader(dat, sampler=val_sampler, batch_size=64)
test_loader = DataLoader(dat, sampler=test_sampler, batch_size=64)

print(f"The training device is: {device}")

Let's now try to make our model better. You can try the following options

1. Make the model convolutional:
    * Instead of a linear layer, use a convolutional network with layers `(Conv2D, Batchnorm2D, Nonlinearity)`.
    * Use a final linear layer + rectifying nonlinearity to map the output of the network to the neurons.
    * A good start is 3 convolutional layers + final linear layer.
    * How many parameters does your model have? Is there a benefit of having many neurons now?
2. Reduce parameters
   * Replace the convolution by depthwise separable convolutions
   * Factorize the readout as in [Klindt et al. 2017](https://arxiv.org/abs/1711.02653).
   * **Advanced**: Implement a Gaussian Readout as in [Lurz et al. 2021](https://openreview.net/pdf?id=Tp7kI90Htd).
3. Improve the training routine
   * Use early stopping: Monitor the validation correlation. If it has not gone up for `patience` epochs, set the model to the best model so far and stop training.
   * Use learning rate schedules: Instead of stopping training, reduce the learning rate once or twice (e.g. with [ReduceLROnPlateau](https://docs.pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html) ).

**Challenge**:
Let's make this a competition. Try to get the best model you can with the above (or other) tricks. We'll maintain a leader board at the front. Report your best validation correlation to us and we mark it down. Obviously, you're not allowed to train on the validation set.

# Visualizing the results

Now that we have a well trained model, we can start analyzing it. However, unlike before, we cannot simply plot linear filters. However, we can run classical analyses (and more) on the model. In the following block we'll look at receptive fields.

1. Run spike triggered averages on the model. I.e. present the model with Gaussian white noise (what previous modelling choice was important for that?) and record the responses. Then, for each if the best 12 neurons, compute the weighted mean of the noise frames using the predicted response of the neuron as weights. Plot the resulting images. What can you observe?

2. What happens if you do 1. with natural images. How can you explain the difference?

3. In stead of using STA, send the zero image through the model and compute the gradient with respect to the image. This can be done via

    ```python
    rfs = []
    for i in best[:12]:
        x = torch.zeros(1, 1, 36, 64).to(device)
        x.requires_grad = True
        r = model_m(x)
        r[0, i].backward()
        rfs.append(x.grad.cpu().numpy().squeeze())
    ```
    
    Plot the resulting images, too. What can you observe? Can you explain why you get this result?

# Optional: How good is good?

The best models so far, probably reached about `0.3` in correlation. Since correlation goes up until `1` this might not seem like a good result. However, neuronal responses are noisy. So even if we had the best predictor, we would not reach `1`. Thus, the question arises what's the best we could do.

For many loss functions, the best predictor is the conditional mean, i.e. the average response to the same image over many repetitions. We can estimate this response from repeated presentations of the same stimulus. The dataset actually contains these kind of repetitions. However, for that we need a new sampler

In [ ]:
 # this sampler groups all responses to the same image in one batch
from neuralpredictors.data.samplers import RepeatsBatchSampler


In [ ]:
root_dir = 'sensorium_data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6'
dat = FileTreeDataset(root_dir, 'images', 'responses')

transforms = [NeuroNormalizer(dat), ScaleInputs(scale=0.25), ToTensor(torch.cuda.is_available())]
dat.transforms.extend(transforms)

test_sampler = RepeatsBatchSampler(dat.trial_info.frame_image_id, np.where(dat.trial_info.tiers == 'test')[0])

test_loader = DataLoader(dat, batch_sampler=test_sampler)

Test that you actually get the same image in each batch

Now use this dataset to compute an "oracle" predictor. Consider the response $y_{ij}$ of a neuron to the $i$th image shown the $j$th time. To compute the oracle predictor for this trial, we average all responses of that neuron to this image over all **but** the $j$th trial

$$\hat y_{ij} = \frac{1}{n-1} \sum_{k\not=j} y_{ik}$$

where $n$ is the number of repeats. Then use this predictor and correlate it with all $y_{ij}$ as if the oracle were model predictions. Compare that to the predictions of the model.